In [14]:
!pip install requests
!pip install BeautifulSoup

  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import os
# URL of the page to scrape
url = 'https://preset.io/blog/'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all <a> tags (links)
    links = soup.find_all('a')

    # Initialize lists to store extracted links
    link_list = []
    filtered_link_list = []

    # Extract and filter links
    for link in links:
        href = link.get('href')
        if href and href.startswith('/'):  # only consider relative URLs
            full_url = urljoin(url, href)  # construct absolute URL
            link_list.append(full_url)

    # Filter links that start with specific URL
    for link in link_list:
        if link.startswith('https://preset.io/blog/'):
            filtered_link_list.append(link)

    # Create DataFrames from the lists of links
    df_all_links = pd.DataFrame(link_list, columns=['Links'])
    df_filtered_links = pd.DataFrame(filtered_link_list, columns=['Filtered Links'])

    # Print the DataFrames
    print("DataFrame of all links:")
    print(df_all_links)
    print("\nDataFrame of filtered links:")
    print(df_filtered_links)

    # Optionally, save the filtered DataFrame to a CSV file
    df_filtered_links.to_csv('filtered_links_from_preset_blog.csv', index=False)

    print(f"Scraped {len(df_filtered_links)} filtered links and saved them to 'filtered_links_from_preset_blog.csv'.")
else:
    print(f"Failed to retrieve content from {url}. Status code: {response.status_code}")


DataFrame of all links:
                                                 Links
0                                   https://preset.io/
1                           https://preset.io/product/
2                           https://preset.io/product/
3      https://preset.io/preset-managed-private-cloud/
4               https://preset.io/embedded-dashboards/
..                                                 ...
216                         https://preset.io/careers/
217           https://preset.io/hiring-referral-bonus/
218                         https://preset.io/contact/
219                  https://preset.io/privacy-policy/
220  https://preset.io/terms-of-service-product-web...

[221 rows x 1 columns]

DataFrame of filtered links:
                                        Filtered Links
0                              https://preset.io/blog/
1    https://preset.io/blog/using-git-bisect-to-fin...
2    https://preset.io/blog/how-and-when-and-why-to...
3    https://preset.io/blog/apache-superse

In [16]:
df_filtered_links

,Filtered Links
0,https://preset.io/blog/
1,https://preset.io/blog/using-git-bisect-to-fin...
2,https://preset.io/blog/how-and-when-and-why-to...
3,https://preset.io/blog/apache-superset-4-0-rel...
4,https://preset.io/blog/exploring-the-dbt-cloud...
...,...
178,https://preset.io/blog/2020-04-15-github-commu...
179,https://preset.io/blog/2020-03-12-corona-virus/
180,https://preset.io/blog/2019-12-16-elasticsearc...
181,https://preset.io/blog/2019-10-09-series-a-ann...


In [17]:
def fetch_and_format_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove unwanted divs
    for div in soup.find_all("div", class_="relative z-10 py-10"):
        div.decompose()
    for div in soup.find_all("div", class_="mt-20 md:text-center"):
        div.decompose()
    for div in soup.find_all("div", class_="nav-mobile"):
        div.decompose()


    text_content = []

    # Fetch and format <h1> and <h> tags
    for header in soup.find_all(['h1', 'h']):
        text_content.append(f"**{header.get_text()}**\n")

    # Fetch and format <p> tags
    for paragraph in soup.find_all('p'):
        text_content.append(f"{paragraph.get_text()}\n")

    # Fetch and format <li> tags
    for list_item in soup.find_all('li'):
        text_content.append(f"• {list_item.get_text()}\n")

    # Fetch and format <code> tags
    for code in soup.find_all('code'):
        text_content.append(f"`{code.get_text()}`\n")

    return text_content

# Create the directory if it doesn't exist
output_dir = "Web_content"
os.makedirs(output_dir, exist_ok=True)

# Iterate through each link in the DataFrame and save the content to a text file
for index, row in df_filtered_links.iterrows():
    url = row['Filtered Links']
    formatted_text = fetch_and_format_text(url)

    # Extract the last hierarchy from the URL path to use as the file name
    file_name = url.rstrip('/').split('/')[-1] + ".txt"
    file_path = os.path.join(output_dir, file_name)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(formatted_text)

    print(f"Content from {url} has been written to {file_path}")

Content from https://preset.io/blog/ has been written to Web_content/blog.txt
Content from https://preset.io/blog/using-git-bisect-to-find-and-fix-bugs/ has been written to Web_content/using-git-bisect-to-find-and-fix-bugs.txt
Content from https://preset.io/blog/how-and-when-and-why-to-update-apache-supersets-country-maps/ has been written to Web_content/how-and-when-and-why-to-update-apache-supersets-country-maps.txt
Content from https://preset.io/blog/apache-superset-4-0-release-notes/ has been written to Web_content/apache-superset-4-0-release-notes.txt
Content from https://preset.io/blog/exploring-the-dbt-cloud-semantic-layer-in-preset/ has been written to Web_content/exploring-the-dbt-cloud-semantic-layer-in-preset.txt
Content from https://preset.io/blog/preset-ai-assist/ has been written to Web_content/preset-ai-assist.txt
Content from https://preset.io/blog/the-preset-api-for-absolute-beginners/ has been written to Web_content/the-preset-api-for-absolute-beginners.txt
Content fr